In [42]:
import pandas as pd
import requests
import numpy as np

In [43]:
data_agg = pd.read_csv("/content/data_agg.csv", sep=',')
raw_data = pd.read_csv("/content/raw_data.csv", sep= ",")

In [44]:
data_agg.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб."
0,0,2022-09-01,3,9,2022,1,0,3,96630.0
1,1,2022-09-02,4,9,2022,1,0,3,108024.5
2,2,2022-09-03,5,9,2022,1,0,3,122352.0
3,3,2022-09-04,6,9,2022,1,0,3,119441.7
4,4,2022-09-05,0,9,2022,1,0,3,87863.5


In [45]:
raw_data.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб."
0,0,2022-09-01,3,9,2022,1,0,3,96630.0
1,1,2022-09-02,4,9,2022,1,0,3,108024.5
2,2,2022-09-03,5,9,2022,1,0,3,122352.0
3,3,2022-09-04,6,9,2022,1,0,3,119441.7
4,4,2022-09-05,0,9,2022,1,0,3,87863.5


In [47]:
raw_data["Открыт_дата"] = pd.to_datetime(raw_data["Открыт_дата"])
raw_data.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб."
0,0,2022-09-01,3,9,2022,1,0,3,96630.0
1,1,2022-09-02,4,9,2022,1,0,3,108024.5
2,2,2022-09-03,5,9,2022,1,0,3,122352.0
3,3,2022-09-04,6,9,2022,1,0,3,119441.7
4,4,2022-09-05,0,9,2022,1,0,3,87863.5


In [48]:
raw_data["День"] = raw_data["Открыт_дата"].dt.day

In [49]:
raw_data.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб.",День
0,0,2022-09-01,3,9,2022,1,0,3,96630.0,1
1,1,2022-09-02,4,9,2022,1,0,3,108024.5,2
2,2,2022-09-03,5,9,2022,1,0,3,122352.0,3
3,3,2022-09-04,6,9,2022,1,0,3,119441.7,4
4,4,2022-09-05,0,9,2022,1,0,3,87863.5,5


In [50]:
data_agg.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб."
0,0,2022-09-01,3,9,2022,1,0,3,96630.0
1,1,2022-09-02,4,9,2022,1,0,3,108024.5
2,2,2022-09-03,5,9,2022,1,0,3,122352.0
3,3,2022-09-04,6,9,2022,1,0,3,119441.7
4,4,2022-09-05,0,9,2022,1,0,3,87863.5


In [51]:
data_agg["Открыт_дата"] = pd.to_datetime(data_agg["Открыт_дата"])
data_agg["День"] = data_agg["Открыт_дата"].dt.day

In [52]:
data_agg.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб.",День
0,0,2022-09-01,3,9,2022,1,0,3,96630.0,1
1,1,2022-09-02,4,9,2022,1,0,3,108024.5,2
2,2,2022-09-03,5,9,2022,1,0,3,122352.0,3
3,3,2022-09-04,6,9,2022,1,0,3,119441.7,4
4,4,2022-09-05,0,9,2022,1,0,3,87863.5,5


In [53]:
def get_temp(year, month, day):

  URL = f"http://www.pogodaiklimat.ru/weather.php?id=27605&bday={day}&fday={day}&amonth={month}&ayear={year}&bot=2"
  data = pd.read_html(URL)
  avg_temp = round(np.mean(pd.to_numeric(data[1][5][2:].values)), 1)

  has_snow = any("снег" in str(item).lower() for item in data[1][3][1:].values)
  has_rain = any("лив" in str(item).lower() or "дождь" in str(item).lower() for item in data[1][3][1:].values)

  if has_snow and has_rain:
      return avg_temp, [1, 1]
  elif has_rain and not has_snow:
      return avg_temp, [0, 1]
  elif has_snow and not has_rain:
      return avg_temp, [1, 0]
  else:
    return avg_temp, [0, 0]









In [54]:
temp_and_conditions = data_agg.apply(lambda row: get_temp(row['Год'], row['Месяц'], row['День']), axis=1)
data_agg['Средняя Температура'] = temp_and_conditions.apply(lambda x: x[0])
data_agg[['Снег', 'Дождь']] = pd.DataFrame(temp_and_conditions.apply(lambda x: x[1]).tolist(), index=data_agg.index)


In [55]:
data_agg.head(20)

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб.",День,Средняя Температура,Снег,Дождь
0,0,2022-09-01,3,9,2022,1,0,3,96630.00,1,11.4,0,0
1,1,2022-09-02,4,9,2022,1,0,3,108024.50,2,10.5,0,1
2,2,2022-09-03,5,9,2022,1,0,3,122352.00,3,12.5,0,0
3,3,2022-09-04,6,9,2022,1,0,3,119441.70,4,12.2,0,0
4,4,2022-09-05,0,9,2022,1,0,3,87863.50,5,10.3,0,0
5,5,2022-09-06,1,9,2022,1,0,3,104438.50,6,9.7,0,1
6,6,2022-09-07,2,9,2022,1,0,3,117891.75,7,9.5,0,1
7,7,2022-09-08,3,9,2022,2,0,3,91541.00,8,9.2,0,1
8,8,2022-09-09,4,9,2022,2,0,3,124681.88,9,10.2,0,0
9,9,2022-09-10,5,9,2022,2,0,3,130622.00,10,12.1,0,0


In [56]:
data_agg.shape

(420, 13)

In [57]:
data_agg.to_csv("data_agg_weather.csv", sep=";")

In [64]:
raw_data = pd.read_csv("/content/raw_data.csv")
raw_data.head()

,Unnamed: 0,Открыт,"Сумма по чеку, руб.",Открыт_дата,Открыт_время,День недели,Месяц,Год,Неделя,ЗП,Время года
0,0,2023-10-26 19:39:00,860.0,2023-10-26,19:39:00,3,10,2023,4,0,3
1,1,2023-10-26 19:37:00,680.0,2023-10-26,19:37:00,3,10,2023,4,0,3
2,2,2023-10-26 19:33:00,810.0,2023-10-26,19:33:00,3,10,2023,4,0,3
3,3,2023-10-26 19:31:00,400.0,2023-10-26,19:31:00,3,10,2023,4,0,3
4,4,2023-10-26 19:26:00,360.0,2023-10-26,19:26:00,3,10,2023,4,0,3


In [65]:
raw_data.shape

(64840, 11)

In [67]:
raw_data["Открыт"] = pd.to_datetime(raw_data["Открыт"])

In [68]:
data_agg["Открыт_дата"] = pd.to_datetime(data_agg["Открыт_дата"])
data_agg.head()

,Unnamed: 0,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб.",День,Средняя Температура,Снег,Дождь
0,0,2022-09-01,3,9,2022,1,0,3,96630.0,1,11.4,0,0
1,1,2022-09-02,4,9,2022,1,0,3,108024.5,2,10.5,0,1
2,2,2022-09-03,5,9,2022,1,0,3,122352.0,3,12.5,0,0
3,3,2022-09-04,6,9,2022,1,0,3,119441.7,4,12.2,0,0
4,4,2022-09-05,0,9,2022,1,0,3,87863.5,5,10.3,0,0


In [69]:
raw_data["День"] = raw_data["Открыт"].dt.day

In [70]:
raw_data.head()

,Unnamed: 0,Открыт,"Сумма по чеку, руб.",Открыт_дата,Открыт_время,День недели,Месяц,Год,Неделя,ЗП,Время года,День
0,0,2023-10-26 19:39:00,860.0,2023-10-26,19:39:00,3,10,2023,4,0,3,26
1,1,2023-10-26 19:37:00,680.0,2023-10-26,19:37:00,3,10,2023,4,0,3,26
2,2,2023-10-26 19:33:00,810.0,2023-10-26,19:33:00,3,10,2023,4,0,3,26
3,3,2023-10-26 19:31:00,400.0,2023-10-26,19:31:00,3,10,2023,4,0,3,26
4,4,2023-10-26 19:26:00,360.0,2023-10-26,19:26:00,3,10,2023,4,0,3,26


In [75]:
data_agg['Открыт_дата'] = pd.to_datetime(data_agg['Открыт_дата'])
raw_data['Открыт_дата'] = pd.to_datetime(raw_data['Открыт_дата'])

In [76]:
raw_data = pd.merge(raw_data, data_agg[['Открыт_дата', 'Средняя Температура', 'Снег', 'Дождь']], on='Открыт_дата', how='left')

In [77]:
raw_data.head()

,Unnamed: 0,Открыт,"Сумма по чеку, руб.",Открыт_дата,Открыт_время,День недели,Месяц,Год,Неделя,ЗП,Время года,День,Средняя Температура,Снег,Дождь
0,0,2023-10-26 19:39:00,860.0,2023-10-26,19:39:00,3,10,2023,4,0,3,26,-0.8,1,0
1,1,2023-10-26 19:37:00,680.0,2023-10-26,19:37:00,3,10,2023,4,0,3,26,-0.8,1,0
2,2,2023-10-26 19:33:00,810.0,2023-10-26,19:33:00,3,10,2023,4,0,3,26,-0.8,1,0
3,3,2023-10-26 19:31:00,400.0,2023-10-26,19:31:00,3,10,2023,4,0,3,26,-0.8,1,0
4,4,2023-10-26 19:26:00,360.0,2023-10-26,19:26:00,3,10,2023,4,0,3,26,-0.8,1,0


In [78]:
raw_data.tail()

,Unnamed: 0,Открыт,"Сумма по чеку, руб.",Открыт_дата,Открыт_время,День недели,Месяц,Год,Неделя,ЗП,Время года,День,Средняя Температура,Снег,Дождь
64835,64835,2022-09-01 08:42:00,1270.0,2022-09-01,08:42:00,3,9,2022,1,0,3,1,11.4,0,0
64836,64836,2022-09-01 08:36:00,450.0,2022-09-01,08:36:00,3,9,2022,1,0,3,1,11.4,0,0
64837,64837,2022-09-01 08:31:00,900.0,2022-09-01,08:31:00,3,9,2022,1,0,3,1,11.4,0,0
64838,64838,2022-09-01 08:26:00,580.0,2022-09-01,08:26:00,3,9,2022,1,0,3,1,11.4,0,0
64839,64839,2022-09-01 08:15:00,310.0,2022-09-01,08:15:00,3,9,2022,1,0,3,1,11.4,0,0


In [79]:
raw_data.to_csv("raw_data_weather.csv", sep =";")